In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from torch_geometric.utils import from_networkx

In [3]:
class DGCNLayer(nn.Module):
    def __init__(self, in_features, out_features, K=2):
        super(DGCNLayer, self).__init__()
        self.K = K  # Число шагов диффузии
        self.weights = nn.ParameterList([
            nn.Parameter(torch.FloatTensor(in_features, out_features))
            for _ in range(K + 1)
        ])
        self.reset_parameters()

    def reset_parameters(self):
        for weight in self.weights:
            nn.init.xavier_uniform_(weight)

    def forward(self, x, P):
        # P — нормированная матрица переходов (D^-1 A)
        out = torch.zeros_like(x)
        P_power = torch.eye(P.size(0))  # P^0 = I

        for k in range(self.K + 1):
            out += torch.mm(P_power, torch.mm(x, self.weights[k]))
            P_power = torch.mm(P_power, P)  # P^k = P^{k-1} * P

        return F.relu(out)

In [4]:
class DGCN(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, K=2):
        super(DGCN, self).__init__()
        self.layer1 = DGCNLayer(num_features, hidden_dim, K)
        self.layer2 = DGCNLayer(hidden_dim, hidden_dim, K)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, P):
        x = self.layer1(x, P)
        x = self.layer2(x, P)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

In [ ]:
""""def visualize_graph(G, labels, title="Graph Structure"):
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, node_color=labels, cmap=plt.cm.tab10,
            node_size=50, with_labels=False)
    plt.title(title)
    plt.colorbar(plt.cm.ScalarMappable(cmap=plt.cm.tab10),
                 label="Node Class")
    plt.show()""""

In [6]:
def main():
    # Параметры
    num_nodes = 100
    num_features = 32
    hidden_dim = 64
    num_classes = 5
    K = 2  # Шаги диффузии
    epochs = 100
    lr = 0.01

    # Генерация графа (Barabasi-Albert)
    G = nx.barabasi_albert_graph(num_nodes, 3)
    features = np.random.randn(num_nodes, num_features)
    labels = np.random.randint(0, num_classes, num_nodes)

    # Матрица переходов P = D^-1 A
    A = torch.FloatTensor(nx.adjacency_matrix(G).toarray())
    D_inv = torch.diag(1.0 / torch.sum(A, dim=1))
    P = torch.mm(D_inv, A)

    # Визуализация графа
  #  visualize_graph(G, labels, "Original Graph")

    # Преобразование данных
    features = torch.FloatTensor(features)
    labels = torch.LongTensor(labels)

    # Разделение данных
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    indices = torch.randperm(num_nodes)
    train_mask[indices[:70]] = True
    val_mask[indices[70:85]] = True
    test_mask[indices[85:]] = True

    # Модель и оптимизатор
    model = DGCN(num_features, hidden_dim, num_classes, K)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.NLLLoss()

    # Обучение
    train_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        out = model(features, P)
        loss = criterion(out[train_mask], labels[train_mask])
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

        # Валидация
        model.eval()
        with torch.no_grad():
            pred = model(features, P).argmax(dim=1)
            val_acc = accuracy_score(labels[val_mask].numpy(), pred[val_mask].numpy())
            val_accuracies.append(val_acc)

        if epoch % 10 == 0:
            print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}")

    # Тестирование
    model.eval()
    with torch.no_grad():
        pred = model(features, P).argmax(dim=1)
        test_acc = accuracy_score(labels[test_mask].numpy(), pred[test_mask].numpy())
        print(f"Test Accuracy: {test_acc:.4f}")

    # Визуализация предсказаний
   # visualize_graph(G, pred.numpy(), "Predicted Classes")

if __name__ == "__main__":
    main()

RuntimeError: The size of tensor a (32) must match the size of tensor b (64) at non-singleton dimension 1

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

class DGCNLayer(nn.Module):
    def __init__(self, in_features, out_features, K=2):
        super(DGCNLayer, self).__init__()
        self.K = K  # Число шагов диффузии
        self.in_features = in_features
        self.out_features = out_features

        # Веса для каждого шага диффузии
        self.weights = nn.ParameterList([
            nn.Parameter(torch.FloatTensor(in_features, out_features))
            for _ in range(K + 1)
        ])
        self.reset_parameters()

    def reset_parameters(self):
        for weight in self.weights:
            nn.init.xavier_uniform_(weight)

    def forward(self, x, P):
        # x: [num_nodes, in_features]
        # P: [num_nodes, num_nodes]
        out = torch.zeros(x.size(0), self.out_features).to(x.device)
        P_power = torch.eye(P.size(0)).to(x.device)  # P^0 = I

        for k in range(self.K + 1):
            # [num_nodes, num_nodes] @ [num_nodes, out_features] = [num_nodes, out_features]
            out += torch.mm(P_power, torch.mm(x, self.weights[k]))
            P_power = torch.mm(P_power, P)  # P^k = P^{k-1} * P

        return F.relu(out)

class DGCN(nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, K=2):
        super(DGCN, self).__init__()
        self.layer1 = DGCNLayer(num_features, hidden_dim, K)
        self.layer2 = DGCNLayer(hidden_dim, num_classes, K)  # Изменено для согласованности размеров

    def forward(self, x, P):
        x = self.layer1(x, P)
        x = self.layer2(x, P)
        return F.log_softmax(x, dim=1)

def compute_diffusion_matrix(G):
    """Вычисляет матрицу диффузии P = D^-1 A"""
    A = nx.adjacency_matrix(G).toarray()
    D_inv = np.diag(1.0 / np.sum(A, axis=1))
    P = np.dot(D_inv, A)
    return torch.FloatTensor(P)

def visualize_graph(G, labels, title="Graph Structure"):
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, node_color=labels, cmap=plt.cm.tab10,
            node_size=50, with_labels=False)
    plt.title(title)
    plt.colorbar(plt.cm.ScalarMappable(cmap=plt.cm.tab10),
                 label="Node Class")
    plt.show()

def main():
    # Параметры
    num_nodes = 100
    num_features = 32
    hidden_dim = 64
    num_classes = 5
    K = 2  # Шаги диффузии
    epochs = 100
    lr = 0.01

    # Генерация графа (Barabasi-Albert)
    G = nx.barabasi_albert_graph(num_nodes, 3)
    features = np.random.randn(num_nodes, num_features)
    labels = np.random.randint(0, num_classes, num_nodes)

    # Матрица диффузии P = D^-1 A
    P = compute_diffusion_matrix(G)

    # Визуализация графа
#    visualize_graph(G, labels, "Original Graph")

    # Преобразование данных
    features = torch.FloatTensor(features)
    labels = torch.LongTensor(labels)

    # Разделение данных
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    indices = torch.randperm(num_nodes)
    train_mask[indices[:70]] = True
    val_mask[indices[70:85]] = True
    test_mask[indices[85:]] = True

    # Модель и оптимизатор
    model = DGCN(num_features, hidden_dim, num_classes, K)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.NLLLoss()

    # Обучение
    train_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        out = model(features, P)
        loss = criterion(out[train_mask], labels[train_mask])
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

        # Валидация
        model.eval()
        with torch.no_grad():
            pred = model(features, P).argmax(dim=1)
            val_acc = accuracy_score(labels[val_mask].numpy(), pred[val_mask].numpy())
            val_accuracies.append(val_acc)

        if epoch % 10 == 0:
            print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}")

    # Тестирование
    model.eval()
    with torch.no_grad():
        pred = model(features, P).argmax(dim=1)
        test_acc = accuracy_score(labels[test_mask].numpy(), pred[test_mask].numpy())
        print(f"Test Accuracy: {test_acc:.4f}")

    # Визуализация предсказаний
#    visualize_graph(G, pred.numpy(), "Predicted Classes")

if __name__ == "__main__":
    main()

Epoch 000, Loss: 1.6684, Val Acc: 0.0667
Epoch 010, Loss: 0.9219, Val Acc: 0.0667
Epoch 020, Loss: 0.8536, Val Acc: 0.0000
Epoch 030, Loss: 0.7718, Val Acc: 0.0667
Epoch 040, Loss: 0.7144, Val Acc: 0.0667
Epoch 050, Loss: 0.7135, Val Acc: 0.0667
Epoch 060, Loss: 0.7132, Val Acc: 0.0667
Epoch 070, Loss: 0.7132, Val Acc: 0.0667
Epoch 080, Loss: 0.7131, Val Acc: 0.0667
Epoch 090, Loss: 0.7130, Val Acc: 0.0667
Test Accuracy: 0.3333
